# 6-7. 프로젝트: 멋진 작사가 만들기

### [루브릭]
평가문항	상세기준
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
  -텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  
      
      
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
  -특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  
     
     
3. 텍스트 생성모델이 안정적으로 학습되었는가?  
  -텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?  
    
### [학습 과정]
1. 데이터 다운로드
2. 데이터 읽어오기
3. 데이터 정제
4. 평가 데이터셋 분리
5. 인공지능 만들기

### [결과 및 회고]

### 데이터 읽어오기

In [1]:
import re                  
import numpy as np         
import tensorflow as tf 
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/E06_LyricistAI/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 읽어오기
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free', 'They come to join us here', 'From sea to shining sea And they all have a dream', 'As people always will', 'To be safe and warm', 'In that shining city on the hill Some wanna slam the door', 'Instead of opening the gate', "Aw, let's turn this thing around", 'Before it gets too late [Chorus]', "It's up to me and you", 'Love can conquer hate', 'I know this to be true', "That's what makes us great [Verse 2]", "Don't tell me a lie", 'And sell it as a fact', "I've been down that road before", "And I ain't goin' back And don't you brag to me", 'That you never read a book', 'I never put my faith', "In a con man and his crooks I won't follow down that path", 'And tempt the hands of fate', "Aw, let's turn this thing around", 'Before it gets too late [Chorus]', "It's up to me and you", 'Love can conquer hate', 'I know this to be true', "That's what makes us great In the quiet of the night", 'I lie here w

### 데이터 정제하기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    #if sentence[0] == "[": continue   # 문장 처음이 '[' 인 문장은 건너뜁니다.
    #if sentence[-1] == "]": continue  # 문장 끝이 ']' 인 문장은 건너뜁니다
    if idx > 30: break   # 문장 30개만 확인
    print(sentence)

[Verse 1]
They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around
Before it gets too late [Chorus]
It's up to me and you
Love can conquer hate
I know this to be true
That's what makes us great [Verse 2]
Don't tell me a lie
And sell it as a fact
I've been down that road before
And I ain't goin' back And don't you brag to me
That you never read a book
I never put my faith
In a con man and his crooks I won't follow down that path
And tempt the hands of fate
Aw, let's turn this thing around
Before it gets too late [Chorus]
It's up to me and you
Love can conquer hate
I know this to be true
That's what makes us great In the quiet of the night
I lie here wide awake
And I ask myself


토큰화

1. 문장 부호 양쪽에 공백을 추가
2. 모든 문자들을 소문자로 변환
3. 특수문자들은 모두 제거

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)       # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)              # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여줌
    
    if "verse" in sentence:
        sentence = sentence.replace("verse", "")
    if "chorus" in sentence:
        sentence = sentence.replace("chorus", "")
    
    
    return sentence


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence) == 1: continue    
    corpus.append(preprocess_sentence(sentence))
corpus.remove('<start>  <end>')         

corpus[:30]
print(len(corpus))

175950


In [34]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있음. 이번에는 사용하지 안함
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 <unk> 처리
    )
    tokenizer.fit_on_texts(corpus)   # 구축한 corpus로부터 Tokenizer가 사전을 자동구축

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환.
   
    for num in tensor:
        if len(num) >= 29:
            tensor = np.delete(tensor, num)
            
    print(len(tensor))
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

    
tensor, tokenizer = tokenize(corpus)

152493
[[   2   45   66 ...    0    0    0]
 [   2    6 1690 ...    0    0    0]
 [   2 4692   16 ...    0    0    0]
 ...
 [   2    7 1019 ...    3    0    0]
 [   2   30   31 ...    0    0    0]
 [   5   32   15 ...   10   12    3]] <keras_preprocessing.text.Tokenizer object at 0x7f97eab76d50>


In [36]:
# 단어 사전 어떻게 생겼는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [38]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성.

print(src_input[0])
print(tgt_input[0])

[  2  45  66  74 802   3   0   0   0   0   0   0   0   0]
[ 45  66  74 802   3   0   0   0   0   0   0   0   0   0]


In [39]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          train_size = 0.8)

In [40]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (121994, 14)
Target Train: (121994, 14)


학습데이터 갯수가 124960개 이하이므로 이대로 학습을 시켜주겠습니다

평가 데이터셋 만들기

In [41]:
BUFFER_SIZE = len(src_input)         
BATCH_SIZE = 256                     
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

인공지능 만들기

In [42]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop  = tf.keras.layers.Dropout(0.5)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.drop(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [43]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[-7.3142830e-05,  1.9110792e-04, -1.2877812e-04, ...,
          1.4936674e-04, -6.2914289e-05,  7.7368190e-05],
        [-3.0964537e-04,  1.0256601e-04, -1.0780464e-04, ...,
          3.5763037e-04, -1.6552306e-04,  2.1371782e-04],
        [-4.6166973e-04,  1.5614412e-04, -1.9737384e-04, ...,
          5.2038813e-04, -3.7981090e-04,  4.0565775e-04],
        ...,
        [-4.8772686e-06, -2.6003379e-04,  6.4323787e-05, ...,
         -4.7925822e-04,  1.5248383e-04,  2.4656547e-04],
        [-1.8396150e-04, -4.3908483e-04,  4.0030832e-04, ...,
         -4.3286735e-04,  6.2110316e-04,  5.9934106e-04],
        [-3.7317694e-04, -6.4272358e-04,  7.5663044e-04, ...,
         -3.9008658e-04,  1.0479168e-03,  9.1990089e-04]],

       [[-7.3142830e-05,  1.9110792e-04, -1.2877812e-04, ...,
          1.4936674e-04, -6.2914289e-05,  7.7368190e-05],
        [ 3.5970126e-05,  2.6167880e-04, -6.3891319e-05, ...,
          3.2870588e-04, 

In [44]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [45]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
595/595 [==============================] - 86s 145ms/step - loss: 3.8058
Epoch 2/30
595/595 [==============================] - 87s 147ms/step - loss: 3.3388
Epoch 3/30
595/595 [==============================] - 88s 147ms/step - loss: 3.1604
Epoch 4/30
595/595 [==============================] - 88s 148ms/step - loss: 3.0328
Epoch 5/30
595/595 [==============================] - 88s 149ms/step - loss: 2.9238
Epoch 6/30
595/595 [==============================] - 89s 149ms/step - loss: 2.8262
Epoch 7/30
595/595 [==============================] - 89s 149ms/step - loss: 2.7360
Epoch 8/30
595/595 [==============================] - 89s 149ms/step - loss: 2.6511
Epoch 9/30
595/595 [==============================] - 89s 149ms/step - loss: 2.5726
Epoch 10/30
595/595 [==============================] - 90s 151ms/step - loss: 2.4973
Epoch 11/30
595/595 [==============================] - 89s 149ms/step - loss: 2.4267
Epoch 12/30
595/595 [==============================] - 87s 146ms/step - lo

In [57]:
results = model.evaluate(enc_val,  dec_val, verbose=2)

print(results)

954/954 - 9s - loss: 1.2473
1.2472774982452393


잘 만들어졌는지 평가하기

In [51]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [52]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you so , <end> '

In [54]:
generate_text(model, tokenizer, init_sentence="<start> i look")

'<start> i look up at the beef , i m a posse <end> '

In [55]:
generate_text(model, tokenizer, init_sentence="<start> that you")

'<start> that you re a liar , a liar <end> '

In [59]:
generate_text(model, tokenizer, init_sentence="<start> what makes")

'<start> what makes the world go round ? <end> '

In [60]:
generate_text(model, tokenizer, init_sentence="<start> hey ")

'<start> hey , hey , hey bobby mcghee , yeah <end> '

In [61]:
generate_text(model, tokenizer, init_sentence="<start> hey you")

'<start> hey you know i got a <unk> like a <unk> <end> '

In [63]:
generate_text(model, tokenizer, init_sentence="<start> If you")

'<start> if you want it , come get it <end> '

## [결과 및 회고]

이번 프로젝트에서는 작사가 AI를 만들어 보았습니다.   
결과적으로, __validation loss = 1.2472774982452393__ 까지 줄이는 모델을 만들 수 있었습니다.   
또한, generate_text 함수를 사용해 실제로 문장이 문맥에 맞게 잘 형성되는지 확인해보았는데,   
생각보다 작곡을 잘 해내서 놀랐습니다.   
아래는 작곡의 예시입니다.    

"< start> what makes"    -> '< start> what makes the world go round ? < end> '   
"< start> i love"        -> '< start> i love you so , < end> '   
"< start> If you"        -> '< start> if you want it , come get it < end> '   


이번 프로젝트에서는 텍스트 데이터 전처리 과정의 중요성에 대해 배울 수 있었습니다.   
전처리 과정에서 정규표현식을 사용하는 방법을 알게 되었고 (아직 익숙하지는 않지만...)
노드에 나와있던 부분 이외에 처리해주어야 할 부분에 대해 더 생각해 볼 수 있었습니다. 저는 __[verse], [chorus]라고 쓰여있는 부분을 제거__ 하는 과정을 추가해주었습니다. 
프로젝트의 목표는 split 하기 전 전체 학습 데이터셋이 124960개 이하가 되도록 전처리하는 것이었는데,  
처음엔 maxlen=15 값을 부여해 padding을 하면 맞출수 있다고 생각했으나 그래도 데이터는 14만개 이상이었습니다.
그래서 padding하기 전, __너무 긴 문장은 제거해주는 과정을 추가__ 해주었고, 목표를 달성할 수 있었습니다.  
